# Pyfolio Analysis - Complete Tearsheet

This notebook demonstrates how to analyze backtest results using pyfolio.

Pyfolio generates comprehensive "tearsheets" including:
- Returns analysis
- Risk metrics (Sharpe, Sortino, Max Drawdown)
- Rolling statistics
- Drawdown periods
- Position concentration
- Transaction costs

## Setup

In [ ]:
# Register Sharadar bundle (required for Jupyter notebooks)
from zipline.data.bundles import register
from zipline.data.bundles.sharadar_bundle import sharadar_bundle

register('sharadar', sharadar_bundle(tickers=None, incremental=True, include_funds=True))
print("✓ Sharadar bundle registered")

In [ ]:
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from zipline import run_algorithm
from zipline.api import (
    order_target_percent,
    symbol,
    record,
    schedule_function,
    date_rules,
    time_rules,
)
from zipline.utils.progress import enable_progress_logging

# Import pyfolio
try:
    import pyfolio as pf
    print("✓ Pyfolio imported successfully")
except ImportError:
    print("⚠️  Pyfolio not installed. Install with: pip install pyfolio-reloaded")
    raise

# Enable logging
logging.basicConfig(level=logging.INFO, force=True)

# Suppress matplotlib.category INFO messages (cosmetic warnings from pyfolio)
logging.getLogger('matplotlib.category').setLevel(logging.WARNING)

enable_progress_logging(algo_name='Pyfolio-Demo', update_interval=20)

## Example Strategy: Momentum Rotation

Rotates between top 3 momentum stocks from a basket of tech stocks.
Rebalances monthly.

In [ ]:
# Universe of stocks to trade
UNIVERSE = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'TSLA', 'NFLX']
TOP_N = 3  # Hold top 3 momentum stocks
MOMENTUM_WINDOW = 60  # 60-day momentum lookback

def initialize(context):
    """
    Initialize strategy.
    """
    # Create symbols
    context.universe = [symbol(s) for s in UNIVERSE]
    context.top_n = TOP_N
    context.momentum_window = MOMENTUM_WINDOW
    
    # Schedule rebalance at start of month
    schedule_function(
        rebalance,
        date_rules.month_start(),
        time_rules.market_open(hours=1)
    )
    
    logging.info(f"Strategy initialized")
    logging.info(f"  Universe: {len(UNIVERSE)} stocks")
    logging.info(f"  Holding top {TOP_N} momentum stocks")
    logging.info(f"  Rebalancing: Monthly")

def rebalance(context, data):
    """
    Monthly rebalance to top momentum stocks.
    """
    # Filter for tradeable stocks only
    tradeable_universe = [stock for stock in context.universe if data.can_trade(stock)]
    
    if len(tradeable_universe) == 0:
        logging.warning("No tradeable stocks in universe")
        return
    
    # Get price history for tradeable stocks
    prices = data.history(
        tradeable_universe,
        'close',
        context.momentum_window + 1,
        '1d'
    )
    
    # Calculate momentum (% change over period)
    momentum = (prices.iloc[-1] / prices.iloc[0]) - 1
    
    # Sort by momentum and get top N (or fewer if not enough tradeable stocks)
    top_n = min(context.top_n, len(tradeable_universe))
    top_stocks = momentum.nlargest(top_n).index
    
    # Equal weight among top stocks
    target_weight = 1.0 / len(top_stocks)
    
    # Rebalance portfolio (only trade tradeable stocks)
    for stock in context.universe:
        if not data.can_trade(stock):
            continue  # Skip untradeable stocks
        
        if stock in top_stocks:
            order_target_percent(stock, target_weight)
        else:
            order_target_percent(stock, 0.0)
    
    # Log current holdings
    holdings = [s.symbol for s in top_stocks]
    logging.info(f"Rebalanced to: {', '.join(holdings)}")

def handle_data(context, data):
    """
    Record daily metrics.
    """
    record(
        portfolio_value=context.portfolio.portfolio_value,
        cash=context.portfolio.cash,
        leverage=context.account.leverage,
    )

## Run Backtest

In [ ]:
# Run backtest
results = run_algorithm(
    start=pd.Timestamp('2019-01-01'),
    end=pd.Timestamp('2023-12-31'),
    initialize=initialize,
    handle_data=handle_data,
    capital_base=100000,
    data_frequency='daily',
    bundle='sharadar',
)

print(f"\n✓ Backtest complete!")
print(f"  Start: {results.index[0].date()}")
print(f"  End: {results.index[-1].date()}")
print(f"  Days: {len(results)}")

## Extract Returns for Pyfolio

Pyfolio requires specific data formats.

In [ ]:
# Extract returns (required for pyfolio)
returns = results['returns']

# Extract positions (optional but recommended)
# KEY INSIGHT: row['positions'] contains ONLY stock positions, NOT cash
# Cash is stored separately in the results DataFrame
positions_data = []

for date, row in results.iterrows():
    pos_dict = {}
    
    # Get cash from results DataFrame (if available)
    if 'cash' in results.columns:
        pos_dict['cash'] = row['cash']
    
    # Iterate through ALL positions (they're all stocks, no cash in this list)
    if row['positions']:
        for pos in row['positions']:
            # Position is a dict with keys: sid, amount, cost_basis, last_sale_price
            sid = pos['sid']
            amount = pos['amount']  # number of shares
            last_sale_price = pos['last_sale_price']  # price per share
            
            # Get symbol from sid (Equity object)
            if hasattr(sid, 'symbol'):
                symbol = sid.symbol
                # Store dollar value (shares * price) for pyfolio
                pos_dict[symbol] = amount * last_sale_price
    
    if pos_dict:
        positions_data.append((date, pos_dict))

if positions_data:
    positions = pd.DataFrame([p[1] for p in positions_data],
                            index=[p[0] for p in positions_data])
    # Fill NaN values with 0 (when a position doesn't exist on certain dates)
    positions = positions.fillna(0)
else:
    positions = None

# Extract transactions (optional)
# Pyfolio expects: index=datetime, columns=[symbol, amount, price, ...]
transactions_list = []
for date, row in results.iterrows():
    if row['transactions']:
        for txn in row['transactions']:
            sid = txn['sid']
            symbol = sid.symbol if hasattr(sid, 'symbol') else str(sid)
            
            transactions_list.append({
                'symbol': symbol,
                'amount': txn['amount'],
                'price': txn['price'],
                'value': txn['amount'] * txn['price'],
            })

if transactions_list:
    transactions = pd.DataFrame(transactions_list,
                               index=[date for date, row in results.iterrows()
                                     if row['transactions'] for _ in row['transactions']])
else:
    transactions = None

print(f"✓ Data prepared for pyfolio")
print(f"  Returns: {len(returns)} days")
if positions is not None:
    print(f"  Positions: {len(positions)} days, {len(positions.columns)} assets ({', '.join(positions.columns)})")
else:
    print(f"  Positions: None")
print(f"  Transactions: {len(transactions) if transactions is not None else 0} trades")
print(f"  Mean daily return: {returns.mean()*100:.3f}%")
print(f"  Daily return std: {returns.std()*100:.3f}%")

## Generate Pyfolio Tearsheet

This creates a comprehensive analysis with:
- Summary statistics
- Worst drawdown periods
- Rolling metrics
- Monthly/yearly returns
- Distribution plots

In [ ]:
# Create full tearsheet
# Note: Set estimate_intraday=False to avoid issues with transaction format detection
pf.create_full_tear_sheet(
    returns,
    positions=positions,
    transactions=transactions,
    live_start_date=None,     # Set if you have live trading data
    round_trips=False,         # Set to True if you want round-trip analysis
    estimate_intraday=False,   # Disable intraday detection to avoid format issues
)

## Custom Analysis: Key Metrics

In [ ]:
# Calculate key metrics manually
annual_return = pf.timeseries.annual_return(returns)
sharpe_ratio = pf.timeseries.sharpe_ratio(returns)
max_drawdown = pf.timeseries.max_drawdown(returns)
sortino_ratio = pf.timeseries.sortino_ratio(returns)
calmar_ratio = pf.timeseries.calmar_ratio(returns)
volatility = pf.timeseries.annual_volatility(returns)

print("\n" + "="*60)
print("KEY PERFORMANCE METRICS")
print("="*60)
print(f"Annual Return: {annual_return*100:.2f}%")
print(f"Annual Volatility: {volatility*100:.2f}%")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Sortino Ratio: {sortino_ratio:.2f}")
print(f"Calmar Ratio: {calmar_ratio:.2f}")
print(f"Max Drawdown: {max_drawdown*100:.2f}%")
print("="*60)

# Calculate cumulative returns
cum_returns = pf.timeseries.cum_returns(returns)
total_return = cum_returns.iloc[-1]
print(f"\nTotal Return: {total_return*100:.2f}%")
print(f"Final Portfolio Value: ${results['portfolio_value'].iloc[-1]:,.2f}")

## Drawdown Analysis

In [ ]:
# Plot drawdowns
fig, ax = plt.subplots(figsize=(14, 6))
pf.plot_drawdown_underwater(returns, ax=ax)
plt.title('Underwater Plot - Drawdown Periods', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Get worst drawdown periods
print("\nTop 5 Drawdown Periods:")
print("="*60)
drawdowns = pf.timeseries.get_top_drawdowns(returns, top=5)

if len(drawdowns) > 0:
    # Calculate drawdown series for looking up values
    cum_returns = pf.timeseries.cum_returns(returns, starting_value=1.0)
    running_max = cum_returns.expanding().max()
    underwater = (cum_returns - running_max) / running_max
    
    for i, dd in enumerate(drawdowns, 1):
        # get_top_drawdowns returns tuples with (peak_date, valley_date, recovery_date)
        peak = pd.Timestamp(dd[0])  # peak_date
        valley = pd.Timestamp(dd[1])  # valley_date
        recovery = pd.Timestamp(dd[2]) if pd.notna(dd[2]) else None  # recovery_date
        
        # Calculate max drawdown for this period
        max_dd = underwater.loc[valley]
        
        recovery_str = recovery.strftime('%Y-%m-%d') if recovery else 'Not recovered'
        
        print(f"{i}. Peak: {peak.strftime('%Y-%m-%d')}, "
              f"Valley: {valley.strftime('%Y-%m-%d')}, "
              f"Recovery: {recovery_str}, "
              f"Drawdown: {max_dd*100:.2f}%")
else:
    print("No significant drawdown periods found.")

## Monthly Returns Heatmap

In [ ]:
# Monthly returns heatmap
fig, ax = plt.subplots(figsize=(12, 6))
pf.plot_monthly_returns_heatmap(returns, ax=ax)
plt.title('Monthly Returns (%)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Rolling Sharpe Ratio

In [ ]:
# Plot rolling Sharpe ratio
fig, ax = plt.subplots(figsize=(14, 6))
pf.plot_rolling_sharpe(returns, ax=ax)
plt.title('Rolling 6-Month Sharpe Ratio', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Returns Distribution

In [ ]:
# Plot returns distribution
fig, ax = plt.subplots(figsize=(10, 6))
pf.plot_return_quantiles(returns, ax=ax)
plt.title('Return Quantiles', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Compare with Benchmark (SPY)

In [ ]:
# Note: For benchmark comparison, you would typically:
# 1. Run a separate backtest with SPY buy-and-hold
# 2. Or fetch SPY returns from data source
# 3. Pass benchmark_rets to create_full_tear_sheet()

# Example structure:
# benchmark_returns = fetch_spy_returns(start_date, end_date)
# pf.create_full_tear_sheet(returns, benchmark_rets=benchmark_returns)

print("\nTo compare with a benchmark:")
print("1. Run a separate buy-and-hold backtest for SPY")
print("2. Extract its returns")
print("3. Pass as benchmark_rets parameter to create_full_tear_sheet()")

## Transaction Analysis

In [ ]:
# Transaction summary using the already-extracted transactions DataFrame
if transactions is not None and len(transactions) > 0:
    print("\n" + "="*60)
    print("TRANSACTION SUMMARY")
    print("="*60)
    print(f"Total Transactions: {len(transactions)}")
    print(f"Buys: {len(transactions[transactions['amount'] > 0])}")
    print(f"Sells: {len(transactions[transactions['amount'] < 0])}")
    print(f"Total Value Traded: ${transactions['value'].abs().sum():,.2f}")
    print(f"Average Trade Size: ${transactions['value'].abs().mean():,.2f}")
    print("\nMost Traded Symbols:")
    print(transactions['symbol'].value_counts().head(5))
    
    # Show sample transactions
    print("\nSample Transactions:")
    print(transactions.head(10).to_string())
else:
    print("\nNo transactions executed.")

## Export Results

Save results for later analysis or reporting.

In [ ]:
# Save to CSV
# results.to_csv('backtest_results.csv')
# returns.to_csv('returns.csv')
# if all_txns:
#     txn_df.to_csv('transactions.csv', index=False)

print("\nTo save results, uncomment the lines above.")

## Summary

**What we covered:**
- Running a momentum rotation strategy
- Generating pyfolio tearsheets
- Analyzing key performance metrics
- Visualizing drawdowns, returns, and risk
- Transaction analysis

**Key pyfolio functions:**
- `create_full_tear_sheet()` - Complete analysis
- `create_returns_tear_sheet()` - Returns focused
- `create_position_tear_sheet()` - Position analysis
- `create_txn_tear_sheet()` - Transaction costs
- `create_round_trip_tear_sheet()` - Round trip analysis

**Next steps:**
- Try with your own strategies
- Compare multiple strategies side-by-side
- Add benchmark comparisons (SPY, QQQ, etc.)
- Analyze factor exposures (requires additional data)